In [2]:
# !unzip '/content/drive/My Drive/Colab Notebooks/cotton_disease/data.zip' -d '/content/drive/My Drive/Colab Notebooks/cotton_disease/'

In [6]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [7]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [30]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from glob import glob

In [29]:
dataset_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/'
training_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/train/'
validation_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/val/'
test_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/test/'
model_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/'

IMAGE_SIZE = [224, 224,3]
EPOCHS = 30
TARGET_SIZE = (224,224)
BATCH_SIZE = 16

In [18]:
# useful for getting number of output classes
folders = glob(training_path+'*')
print(len(folders))

4


In [11]:
# importing the Inception V3 and using the imagenet weights
# Here we will be using imagenet weights

inception = InceptionV3(include_top=False,weights='imagenet',input_shape=IMAGE_SIZE)

87916544/87910968 [==============================] - 0s 0us/step


In [12]:
# don't train existing weights
for layer in inception.layers:
  layer.trainable = False

In [19]:
# our layers - you can add more if you want
x_network = Flatten()(inception.output)
# adding the prediction layer
prediction_layer = Dense(len(folders),activation='softmax')(x_network)

In [20]:
# create a model object
model = Model(inputs = inception.input , outputs = prediction_layer)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [21]:
# compiling the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [33]:
# creating callbacks to save best model
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath= model_path,
                                                               save_weights_only='False',
                                                               monitor='val_acc',
                                                               save_best_only=True)


In [32]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('acc') > 0.99):
      print('Deserved accuracy reached.....!!')
      self.model.stop_training = True
  

In [23]:
# creating datagenerators

# train datagenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# test datagenerator
test_datagen = ImageDataGenerator(rescale= 1./255)

In [28]:
training_data = train_datagen.flow_from_directory(training_path,
                                                  target_size = TARGET_SIZE,
                                                  batch_size = BATCH_SIZE,
                                                  class_mode = 'categorical')

validation_dataset = test_datagen.flow_from_directory(validation_path,
                                                   target_size = TARGET_SIZE,
                                                   batch_size = BATCH_SIZE,
                                                   class_mode = 'categorical')

Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.


In [34]:
# fit the model

r = model.fit_generator(training_data,
                        validation_data = validation_dataset,
                        epochs = EPOCHS,
                        steps_per_epoch = len(training_data),
                        validation_steps = len(validation_dataset),
                        callbacks=[model_checkpoint_callback , MyCallback()])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
116/122 [===========================>..] - ETA: 1s - loss: 2.0657 - accuracy: 0.7585

KeyboardInterrupt: ignored